In [4]:
import requests, re
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time, random
from urllib.request import urlopen
import urllib.parse
import numpy as np
from IPython.display import display
import datetime
# from Levenshtein import distance as lev
import Levenshtein as lev
from itertools import product
# from scrapeFunc import *

In [15]:
lev.ratio('harry potter', 'potter harry')

0.5

In [260]:
def start_df_lev(path, levscore, start_row=0):
    df = pd.read_csv(path, index_col=0)
    df = df[(df.author.notna()) & (df.overdrive_author.notna())]
    df["LevScore"] = df.apply(lambda x: lev.ratio(x.author ,x.overdrive_author), axis=1)
    df.loc[(df.LevScore < levscore), ['title', 'author', 'author_url', 'stars',
           'number of ratings', 'url', 'is_series', 'year_published', 'price',
           'number of pages', 'is_movie', 'movie title', 'movie year','author_stars', 'author_ratings',
           'author_num_reviews', 'author_distinct_works']] = np.nan
    list_of_indices = (list(df[df.author.isna()].index))
    list_of_indices = [row for row in list_of_indices if row >= start_row]
#     print(list_of_indices)
    
    return list_of_indices, df

In [324]:
def restart_df_lev(path, levscore, start_row):
    df = pd.read_csv(path, index_col=0)
#     df = df[(df.author.notna()) & (df.overdrive_author.notna())]
#     df["LevScore"] = df.apply(lambda x: lev.ratio(x.author ,x.overdrive_author), axis=1)
#     df.loc[(df.LevScore < levscore), ['title', 'author', 'author_url', 'stars',
#            'number of ratings', 'url', 'is_series', 'year_published', 'price',
#            'number of pages', 'is_movie', 'movie title', 'movie year','author_stars', 'author_ratings',
#            'author_num_reviews', 'author_distinct_works']] = np.nan
    list_of_indices = (list(df[df.author.isna()].index))
    list_of_indices = [row for row in list_of_indices if row >= start_row]
#     print(list_of_indices)
    
    return list_of_indices, df

In [235]:
imported = start_df_lev('Figures/merged_df_lev.csv', 0.43, 200)

In [236]:
imported[0]

[227,
 229,
 248,
 269,
 271,
 289,
 293,
 313,
 315,
 337,
 339,
 355,
 357,
 359,
 360,
 365,
 367,
 430,
 432,
 450,
 452,
 462,
 464,
 474,
 476,
 499,
 501,
 529,
 531,
 543,
 545,
 563,
 565,
 583,
 585,
 592,
 594,
 644,
 646,
 680,
 682,
 748,
 750,
 772,
 774,
 789,
 791,
 806,
 808,
 810,
 812,
 826,
 828,
 834,
 836,
 858,
 860,
 879,
 881,
 883,
 885,
 900,
 902,
 908,
 917,
 919,
 951,
 953,
 963,
 965,
 975,
 979,
 999,
 1001,
 1024,
 1026,
 1045,
 1047,
 1065,
 1067,
 1077,
 1079,
 1093,
 1095,
 1106,
 1108,
 1134,
 1136,
 1142,
 1144,
 1214,
 1216,
 1279,
 1281,
 1287,
 1289,
 1320,
 1398,
 1400,
 1427,
 1429,
 1476,
 1478,
 1532,
 1536,
 1630,
 1634,
 1726,
 1833,
 1837,
 1906,
 1910,
 2066,
 2070,
 2074,
 2078,
 2166,
 2170,
 2207,
 2211,
 2235,
 2237,
 2239,
 2241,
 2279,
 2281,
 2301,
 2303,
 2381,
 2383,
 2402,
 2404,
 2461,
 2463,
 2470,
 2472,
 2522,
 2524,
 2526,
 2528,
 2534,
 2536,
 2542,
 2544,
 2546,
 2548,
 2583,
 2585,
 2595,
 2597,
 2599,
 2601,
 2694,
 2

In [228]:
path = 'Figures/merged_df_lev.csv'
levscore = 0.43
df = pd.read_csv(path, index_col=0)

In [187]:
df = df[(df.author.notna()) & (df.overdrive_author.notna())]

In [188]:
df["LevScore"] = df.apply(lambda x: lev.ratio(x.author ,x.overdrive_author), axis=1)

In [189]:
df.loc[(df.LevScore < levscore) & (df.author != '-1'), ['title', 'author', 'author_url', 'stars',
           'number of ratings', 'url', 'is_series', 'year_published', 'price',
           'number of pages', 'is_movie', 'movie title', 'movie year','author_stars', 'author_ratings',
           'author_num_reviews', 'author_distinct_works']] = np.nan

In [166]:
df.to_csv('Figures/test.csv')

In [312]:
def get_pages_year_authorURL_GR_attr_redo_Lev(df, index): # pass in df, func adds info, returns mod df
    
    base_url = 'https://www.goodreads.com'
    results = {}

    
#         soup = get_soup_proxy(base_url + df.loc[index, 'url'])
    if df.loc[index, 'url'] == -1 or df.loc[index, 'url'] == '-1': 
            
        results['author_stars'] = -1      
        results['author_ratings'] = -1
        results['author_num_reviews'] = -1   
        results['author_distinct_works'] = -1
        results['number of pages'] = -1
        results['year_published'] = -1
        return results
        
        
        
        
        
        
    print(df.loc[index, 'url'])   
    soup = get_soup_driver(base_url + df.loc[index, 'url'])        

    try: num_pages = soup.find(itemprop='numberOfPages').text
    except: num_pages = -1

    try: year_published = soup.find(text = re.compile('Published')).text.replace('Published','').replace('\n', '').strip()
    except: year_published = -1

    try: author_url = df.loc[index, 'author_url']
    except: author_url = -1

#     print(num_pages, year_published, 'num pages', 'year_published')

    time.sleep(20 + random.uniform(-1,1))

#         if author_url != -1: soup = get_soup_proxy(author_url)
    if author_url != -1: 
        print('authorURL', author_url)
        soup = get_soup_driver(author_url)

        try: 
            rating = soup.find(class_='rating').text.replace('Average rating:', '').replace('\n','').strip()#replace
            if rating is None:
                raise Exception
        except: rating = -1

        try: 
            num_ratings = soup.find(class_='votes').text.replace('\n','').strip()#replace
            if num_ratings is None:
                raise Exception
        except: num_ratings = -1

        try: 
            num_reviews = soup.find(class_='count').text.replace('\n','').strip()#replace
            if num_reviews is None:
                raise Exception
        except: num_reviews = -1

        try: 
            num_distinct_works = soup.find(class_='hreview-aggregate').find('a').text
            if num_distinct_works is None:
                raise Exception
        except: num_distinct_works = -1           

        print(rating, num_ratings, num_reviews, num_distinct_works)
#         print('authorURL', author_url)
        results['author_stars'] = rating      
        results['author_ratings'] = num_ratings
        results['author_num_reviews'] = num_reviews   
        results['author_distinct_works'] = num_distinct_works
        results['number of pages'] = num_pages
        results['year_published'] = year_published

        
#         df.to_csv(path)
        time.sleep(20 + random.uniform(-1,1))
        return results
#     return pd.DataFrame(list_results)
    

In [313]:
def search_goodreads_book_info(df, index, title, author): # returns a dict containing titles, ratings, urls, date published
    base_url = 'https://www.goodreads.com/search?q='
    url = base_url + urllib.parse.quote(title + ' ' + author, safe='~')
    print(url)
    search_results = scrape_goodreads(url)
#     print(search_results)
#  
    results = {}
    
    try: 
        row_number = search_results['number of ratings'].apply(lambda x: int(x.split()[-2].replace(',', ''))).idxmax()
        title = search_results.loc[row_number, 'title']
        author = search_results.loc[row_number, 'author']
        author_url = search_results.loc[row_number, 'author_url']    
        author_url = author_url[:author_url.index('?')]
        url = search_results.loc[row_number, 'url'] 
        url = url[:url.index('?')]
        stars = search_results.loc[row_number, 'stars']    
        is_series = search_results.loc[row_number, 'is_series'] 
        num_ratings = search_results.loc[row_number, 'number of ratings'] 

        

    except Exception as e:
        title = -1
        author = -1
        author_url = -1
        url = -1 
        stars = -1
        is_series = -1
        num_ratings = -1
        

        
#         print(e)
                         
    results['title'] = title
    results['author'] = author
    results['author_url'] = author_url
    results['url'] = url
    results['stars'] = stars
    results['is_series'] = is_series
    results['number of ratings'] = num_ratings
    
    df.loc[index, 'title'] = results['title']
    df.loc[index, 'author'] = results['author']
    df.loc[index, 'author_url'] = results['author_url']
    df.loc[index, 'url'] = results['url']
    df.loc[index, 'stars'] = results['stars']
    df.loc[index, 'is_series'] = results['is_series']

    
    results.update(get_pages_year_authorURL_GR_attr_redo_Lev(df, index))
    
#     print(results)
#     print(title)
    return results

In [330]:
def put_in_book_info(path, levscore, start_row): # returns a df containing titles, ratings, urls, date published
    global driver
    driver = webdriver.Chrome()
    list_of_indices, df = restart_df_lev(path, levscore, start_row)
#     print(list_of_indices)
    for index in list_of_indices:
        title = df.loc[index, 'Overdrive title']
        author = df.loc[index, 'overdrive_author']
        try:
            results = search_goodreads_book_info(df, index, title, author)
    #             df.loc[index, 'title'] = results['title']
    #             df.loc[index, 'author'] = results['author']
    #             df.loc[index, 'author_url'] = results['author_url']
    #             df.loc[index, 'url'] = results['url']
    #             df.loc[index, 'stars'] = results['stars']
    #             df.loc[index, 'is_series'] = results['is_series']
            df.loc[index, 'number of ratings'] = results['number of ratings']
            df.loc[index, 'year_published'] = results['year_published']
            df.loc[index, 'number of pages'] = results['number of pages']
            df.loc[index, 'author_stars'] = results['author_stars']
            df.loc[index, 'author_ratings'] = results['author_ratings']
            df.loc[index, 'author_num_reviews'] = results['author_num_reviews']
            df.loc[index, 'author_distinct_works'] = results['author_distinct_works']
            df.loc[index, 'url'] = results['url']

        except Exception as e:
            print(e)
            driver.quit()
            time.sleep(20)
            driver = webdriver.Chrome()
            continue
            
        df.to_csv(path)
#         time.sleep(20)
    driver_proxy.quit()


In [331]:
put_in_book_info('Figures/merged_df_lev.csv', 0.43, 0)

https://www.goodreads.com/search?q=Persuasion%20-1
substring not found
https://www.goodreads.com/search?q=Persuasion%20-1
substring not found
https://www.goodreads.com/search?q=The%20Raven%20Boys%20-1
/book/show/17675462-the-raven-boys
authorURL https://www.goodreads.com/author/show/1330292.Maggie_Stiefvater
3.98 1,828,631 167,667 50 distinct works
https://www.goodreads.com/search?q=The%20Raven%20Boys%20-1
/book/show/17675462-the-raven-boys
authorURL https://www.goodreads.com/author/show/1330292.Maggie_Stiefvater
3.98 1,828,631 167,667 50 distinct works
https://www.goodreads.com/search?q=The%20Truth%20About%20Forever%20-1
/book/show/34564342-the-truth-about-forever
authorURL https://www.goodreads.com/author/show/16568100.Cole_Lepley
4.04 785 360 10 distinct works
https://www.goodreads.com/search?q=The%20Truth%20About%20Forever%20-1
/book/show/34564342-the-truth-about-forever
authorURL https://www.goodreads.com/author/show/16568100.Cole_Lepley
4.04 785 360 10 distinct works
https://www.

/book/show/40245968-brooklyn-bombshells---part-1
authorURL https://www.goodreads.com/author/show/2284216.Erica_Hilton
4.37 918 135 17 distinct works
https://www.goodreads.com/search?q=Black%20Beauty%20Erica%20Hilton
/book/show/40245968-brooklyn-bombshells---part-1
authorURL https://www.goodreads.com/author/show/2284216.Erica_Hilton
4.37 918 135 17 distinct works
https://www.goodreads.com/search?q=To%20Live%20Again%20and%20The%20Second%20Trip%20Robert%20Silverberg
/book/show/17859464-to-live-again-and-the-second-trip
authorURL https://www.goodreads.com/author/show/4338.Robert_Silverberg
3.88 213,778 15,203 1,701 distinct works
https://www.goodreads.com/search?q=Skies%20Discrowned%20and%20An%20Epitaph%20in%20Rust%20Robert%20Silverberg
https://www.goodreads.com/search?q=Beowulf%20Maria%20Dahvana%20Headley
/book/show/41940267-beowulf
authorURL https://www.goodreads.com/author/show/144323.Maria_Dahvana_Headley
3.71 54,104 10,356 83 distinct works
https://www.goodreads.com/search?q=Beowulf%2

/book/show/52806054-this-side-of-paradise---the-original-1920-edition
authorURL https://www.goodreads.com/author/show/28242.Jackson_R_Bryer
4.06 3,608 266 59 distinct works
https://www.goodreads.com/search?q=Remembrance%20-1
/book/show/20518872-the-three-body-problem
authorURL https://www.goodreads.com/author/show/5780686.Liu_Cixin
4.19 396,475 37,729 109 distinct works
https://www.goodreads.com/search?q=Remembrance%20Danielle%20Steel
/book/show/59856.Remembrance
authorURL https://www.goodreads.com/author/show/14255.Danielle_Steel
3.92 1,056,879 59,030 350 distinct works
https://www.goodreads.com/search?q=Defending%20Jacob%20-1
/book/show/42650213-protect-defend
authorURL https://www.goodreads.com/author/show/8305779.Alex_Ander
4.28 637 56 29 distinct works
https://www.goodreads.com/search?q=Dead%20in%20the%20Family%20-1
/book/show/7091488-dead-in-the-family
authorURL https://www.goodreads.com/author/show/17061.Charlaine_Harris
3.93 3,531,137 135,796 206 distinct works
https://www.good

authorURL https://www.goodreads.com/author/show/43393.Danielle_Trussoni
3.44 25,516 4,042 11 distinct works
https://www.goodreads.com/search?q=The%20Ancestor%20Danielle%20Trussoni
/book/show/46159088-the-ancestor
authorURL https://www.goodreads.com/author/show/43393.Danielle_Trussoni
3.44 25,516 4,042 11 distinct works
https://www.goodreads.com/search?q=Whiskey%20Sharp%20Lauren%20Dane
/book/show/31394318-unraveled
authorURL https://www.goodreads.com/author/show/226816.Lauren_Dane
3.96 200,238 13,439 103 distinct works
https://www.goodreads.com/search?q=Torn%20Jennifer%20L.%20Armentrout
/book/show/31038172-torn
authorURL https://www.goodreads.com/author/show/4476934.Jennifer_L_Armentrout
4.21 2,720,810 236,907 103 distinct works
https://www.goodreads.com/search?q=After%20Bruce%20Greyson%2C%20M.D.
https://www.goodreads.com/search?q=Island%20Jason%20Chin
/book/show/13170025-island
authorURL https://www.goodreads.com/author/show/727043.Jason_Chin
4.25 14,308 3,189 17 distinct works
https:/

/book/show/37186.The_Miraculous_Journey_of_Edward_Tulane
authorURL https://www.goodreads.com/author/show/13663.Kate_DiCamillo
4.07 739,995 58,977 85 distinct works
https://www.goodreads.com/search?q=Ruthless%20River%20Holly%20FitzGerald
/book/show/33517046-ruthless-river
authorURL https://www.goodreads.com/author/show/16210279.Holly_Conklin_FitzGerald
4.03 2,378 308 1 distinct work
https://www.goodreads.com/search?q=And%20the%20Shofar%20Blew%20Francine%20Rivers
/book/show/95619.And_the_Shofar_Blew
authorURL https://www.goodreads.com/author/show/6492.Francine_Rivers
4.38 744,204 36,749 61 distinct works
https://www.goodreads.com/search?q=Demons%20Bill%20Pronzini
/book/show/993059.Demons
authorURL https://www.goodreads.com/author/show/22880.Bill_Pronzini
3.68 27,514 4,453 490 distinct works
https://www.goodreads.com/search?q=Truly%20Ruthie%20Knox
/book/show/18481904-truly
authorURL https://www.goodreads.com/author/show/5308032.Ruthie_Knox
3.87 36,266 5,839 32 distinct works
https://www.g

3.98 25,743 5,126 36 distinct works
https://www.goodreads.com/search?q=Confessions%20Cynthia%20Eden
/book/show/22716120-confessions
authorURL https://www.goodreads.com/author/show/153738.Cynthia_Eden
4.05 236,112 20,571 197 distinct works
https://www.goodreads.com/search?q=Shadows%20of%20My%20Father%20Christoph%20Werner
/book/show/34848250-shadows-of-my-father
authorURL https://www.goodreads.com/author/show/733665.Christoph_Werner
3.44 16 3 25 distinct works
https://www.goodreads.com/search?q=Shadow%27s%20Son%20Jon%20Sprunk
/book/show/6951425-shadow-s-son
authorURL https://www.goodreads.com/author/show/2915851.Jon_Sprunk
3.65 9,974 665 19 distinct works
https://www.goodreads.com/search?q=Founding%20Feuds%20Paul%20Aron
/book/show/27173064-founding-feuds
authorURL https://www.goodreads.com/author/show/242825.Paul_Aron
3.41 1,373 207 18 distinct works
https://www.goodreads.com/search?q=Someone%20Like%20You%20Karen%20Kingsbury
/book/show/52271365-someone-like-you
authorURL https://www.good

authorURL https://www.goodreads.com/author/show/64103.Richard_Ives
3.99 68 12 7 distinct works
https://www.goodreads.com/search?q=Real%20Carol%20Cujec
/book/show/51253501-real
authorURL https://www.goodreads.com/author/show/20018292.Carol_Cujec
4.6 514 185 1 distinct work
https://www.goodreads.com/search?q=Harlequin%20Presents%2C%20November%202019%2C%20Box%20Set%202%20of%202%20Sharon%20Kendrick
/book/show/44279061-harlequin-presents---november-2019---box-set-2-of-2
authorURL https://www.goodreads.com/author/show/181368.Sharon_Kendrick
3.48 26,682 2,916 778 distinct works
https://www.goodreads.com/search?q=Kept%20by%20the%20Spanish%20Billionaire%20Cathy%20Williams
/book/show/6283953-kept-by-the-spanish-billionaire
authorURL https://www.goodreads.com/author/show/4992.Cathy_Williams
3.41 17,167 1,660 793 distinct works
https://www.goodreads.com/search?q=In%20the%20Country%20of%20the%20Blind%20Edward%20Hoagland
/book/show/28695413-in-the-country-of-the-blind
authorURL https://www.goodreads

4.06 170,333 21,110 165 distinct works
https://www.goodreads.com/search?q=Back%20to%20You%20Chris%20Scully
/book/show/33983712-back-to-you
authorURL https://www.goodreads.com/author/show/6152322.Chris_Scully
3.78 5,032 1,017 18 distinct works
https://www.goodreads.com/search?q=The%20Dragon%27s%20Eye%20Sarwat%20Chadda
/book/show/36127361-the-dragon-s-eye
authorURL https://www.goodreads.com/author/show/2869550.Sarwat_Chadda
3.92 15,489 2,372 46 distinct works
https://www.goodreads.com/search?q=Love%20Unleashed%20Lori%20Foster
/book/show/14927035-love-bites
authorURL https://www.goodreads.com/author/show/17597.Lori_Foster
4.02 350,613 23,717 311 distinct works
https://www.goodreads.com/search?q=The%20Great%20Alone%20Janet%20Dailey
/book/show/22706446-the-great-alone
authorURL https://www.goodreads.com/author/show/30669.Janet_Dailey
3.98 76,006 5,541 241 distinct works
https://www.goodreads.com/search?q=The%20Captive%20Elle%20Kennedy
/book/show/45175139-the-captive
authorURL https://www.go

authorURL https://www.goodreads.com/author/show/9833.Iain_Pears
3.83 56,054 4,908 20 distinct works
https://www.goodreads.com/search?q=The%20Synapse%20Sequence%20Daniel%20Godfrey
/book/show/38104467-the-synapse-sequence
authorURL https://www.goodreads.com/author/show/3930960.Daniel_Godfrey
3.43 1,193 198 4 distinct works
https://www.goodreads.com/search?q=La%20Cronopandilla%20Ana%20Campoy
/book/show/38613515-el-t-nel-del-tiempo
authorURL https://www.goodreads.com/author/show/6876264.Ana_Campoy
4.03 525 62 23 distinct works
https://www.goodreads.com/search?q=Another%20Place%20You%27ve%20Never%20Been%20Rebecca%20Kauffman
/book/show/28819025-another-place-you-ve-never-been
authorURL https://www.goodreads.com/author/show/14931785.Rebecca_Kauffman
3.58 8,783 1,230 5 distinct works


In [249]:
driver = webdriver.Chrome()

In [272]:
author_info = soup.find_all('a', class_='authorName')
authors = [author.text for author in author_info]
author_urls = [author.get('href') for author in author_info]
author_urls

[]

In [120]:
author_urls[3][:author_urls[3].index('?')]

'https://www.goodreads.com/author/show/3915.Steven_Pinker'

In [98]:
soup_titles = soup.find_all('a', class_='bookTitle')
titles, urls = [(title.text.strip(), title.get('href')) for title in soup_titles ]

ValueError: too many values to unpack (expected 2)

In [114]:
dates = soup.find_all(class_='minirating')
# dates = [date.parent.text.split() for date in dates]
dates = [date.parent.text for date in dates]
dates[3]

'\n 3.89 avg rating — 1,396 ratings\n              —\n                published\n               2021\n              —\n              26 editions\n'

In [158]:
import scraper_functions

In [159]:
scraper_functions.test()

sdf


In [163]:
test = pd.read_csv('Figures/merged_df.csv')

In [165]:
test = test.iloc[:2690]
test[test.year_published.notna()]

,Unnamed: 0,id_number,title,author,author_url,stars,number of ratings,url,is_series,year_published,...,Wait time,Library copies,People waiting in total,People waiting per copy,Genres,overdrive_author,author_stars,author_ratings,author_num_reviews,author_distinct_works
0,0,0.0,Night,Elie Wiesel,https://www.goodreads.com/author/show/1049.Eli...,4.34 avg rating,"1,076,237 ratings",/book/show/1617.Night,True,1956,...,0,5,-1,0,"['Biography & Autobiography', 'History', 'Nonf...",Elie Wiesel,4.32,"1,136,088","36,845",203 distinct works
1,2,0.0,Night,Elie Wiesel,https://www.goodreads.com/author/show/1049.Eli...,4.34 avg rating,"1,076,237 ratings",/book/show/1617.Night,True,1956,...,0,7,-6,0,"['Biography & Autobiography', 'Classic Literat...",Elie Wiesel,4.32,"1,136,088","36,845",203 distinct works
2,4,1.0,The Westing Game,Ellen Raskin,https://www.goodreads.com/author/show/10074.El...,4.00 avg rating,"195,985 ratings",/book/show/902.The_Westing_Game,False,1978,...,About 3 weeks,8,8,1,"['Juvenile Fiction', 'Juvenile Literature', 'M...",Ellen Raskin,4.00,"201,247","12,487",40 distinct works
3,6,1.0,The Westing Game,Ellen Raskin,https://www.goodreads.com/author/show/10074.El...,4.00 avg rating,"195,985 ratings",/book/show/902.The_Westing_Game,False,1978,...,About 2 weeks,4,2,< 1,"['Juvenile Fiction', 'Juvenile Literature', 'M...",Ellen Raskin,4.00,"201,247","12,487",40 distinct works
4,8,2.0,Hunted,P.C. Cast,https://www.goodreads.com/author/show/17015.P_...,3.91 avg rating,"178,631 ratings",/book/show/4134071-hunted,True,"March 14, 2017",...,0,2,-2,0,"['Comic and Graphic Books', 'Fiction']",Nick Spencer,3.93,"2,274,202","64,083",108 distinct works
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2683,5263,1619.0,The Fifth Season,N.K. Jemisin,https://www.goodreads.com/author/show/2917917....,4.28 avg rating,"203,855 ratings",/book/show/19161852-the-fifth-season,True,"August 4, 2015",...,Available soon,56,20,< 1,"['Fantasy', 'Fiction', 'Literature', 'Thriller']",N. K. Jemisin,NaN,NaN,NaN,NaN
2684,5264,1619.0,The Fifth Season,N.K. Jemisin,https://www.goodreads.com/author/show/2917917....,4.28 avg rating,"203,855 ratings",/book/show/19161852-the-fifth-season,True,"August 4, 2015",...,info unavailable,info unavailable,info unavailable,info unavailable,"['Fantasy', 'Fiction', 'Literature']",N. K. Jemisin,NaN,NaN,NaN,NaN
2685,5265,1620.0,El túnel,Ernesto Sabato,https://www.goodreads.com/author/show/5775606....,4.04 avg rating,"54,671 ratings",/book/show/53447.El_t_nel,False,1948,...,0,2,-1,0,"['Fiction', 'Literature']",Ernesto Sabato,NaN,NaN,NaN,NaN
2686,5266,1620.0,El túnel,Ernesto Sabato,https://www.goodreads.com/author/show/5775606....,4.04 avg rating,"54,671 ratings",/book/show/53447.El_t_nel,False,1948,...,0,1,-1,0,"['Juvenile Fiction', 'Juvenile Literature', 'S...",Ana Campoy,NaN,NaN,NaN,NaN


In [264]:
def get_soup_proxy_(url): # Retrieves soup from website using py requests module and 'person' headers
    API_KEY = 'OavaC96DqSNsO1vM2qStOhzTZoeOakNd'
    base_url = f'https://api.webscrapingapi.com/v1?api_key={API_KEY}&url='
    url = base_url + urllib.parse.quote(url, safe='~')
    response = requests.get(url, timeout=(10,20))
    status = response.status_code
    if status == 200:
        page = response.text
        soup = bs(page)
    else:
        print(f"Oops! Received status code {status}")
    return soup

# def get_soup_proxy(url): # Retrieves soup from website using py requests module and 'person' headers
# #     API_KEY = 'OavaC96DqSNsO1vM2qStOhzTZoeOakNd'
# #     base_url = f'https://api.webscrapingapi.com/v1?api_key={API_KEY}&url='
# #     url = base_url + urllib.parse.quote(url, safe='~')
# #     response = requests.get(url, timeout=(10,20))
# #     status = response.status_code
    
#     url = "https://api.webscrapingapi.com/v1"

#     params = {
#       "api_key":"OavaC96DqSNsO1vM2qStOhzTZoeOakNd",
#       "url":url
#     }
#     response = requests.request("GET", url, params=params, timeout=(10,20))
#     status = response.status_code

#     if status == 200:
#         page = response.text
#         soup = bs(page)
#     else:
#         print(f"Oops! Received status code {status}")
#     return soup

In [27]:
scrapeFunc.test()

NameError: name 'scrapeFunc' is not defined

In [66]:
#!/usr/bin/env python
# coding: utf-8

# In[2]:


# import requests, re
# from bs4 import BeautifulSoup as bs
# import pandas as pd
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# import time, random
# from urllib.request import urlopen
# import urllib.parse
# import numpy as np
# from IPython.display import display


# In[5]:


def get_soup(url): # Retrieves soup from website using py requests module and 'person' headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:95.0) Gecko/20100101 Firefox/95.0',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Referer': 'https://duckduckgo.com/'
        }
    response = requests.get(url, headers=headers, timeout=(10,20))
    status = response.status_code
    if status == 200:
      page = response.text
      soup = bs(page)
    else:
      print(f"Oops! Received status code {status}")
    return soup


# In[13]:


def get_soup_proxy(url): # Retrieves soup from website using py requests module and 'person' headers
    print('ding')
    API_KEY = 'OavaC96DqSNsO1vM2qStOhzTZoeOakNd'
    base_url = f'https://api.webscrapingapi.com/v1?api_key={API_KEY}&url='
    url = base_url + urllib.parse.quote(url, safe='~')
    response = requests.get(url, timeout=(10,20))
    status = response.status_code
    
    # url = "https://api.webscrapingapi.com/v1"

    # params = {
    #   "api_key":"OavaC96DqSNsO1vM2qStOhzTZoeOakNd",
    #   "url":url
    # }
    # response = requests.request("GET", url, params=params, timeout=(10,20))
    # status = response.status_code

    if status == 200:
        page = response.text
        soup = bs(page)
    else:
        print(f"Oops! Received status code {status}")
    return soup


# In[6]:


def get_soup_driver(base_url): # Retrieves soup from website using Selenium
#     driver = webdriver.Chrome()
    driver.get(base_url)
    time.sleep(10 + random.uniform(-1,1))
    r = driver.page_source
    
#     driver.quit()
    return bs(r, 'html.parser')  


# In[8]:


def get_soup_driver_proxy(base_url): # Retrieves soup from website using Selenium
#     driver = webdriver.Chrome()
    YOUR_API_KEY = 'OavaC96DqSNsO1vM2qStOhzTZoeOakNd'
    target_url = urllib.parse.quote(base_url, safe='~')

    URL = f"https://api.webscrapingapi.com/v1/?api_key={YOUR_API_KEY}&url=" + target_url

    driver_proxy.get(URL)
    time.sleep(2 + random.uniform(-1,1))

    # check if accept cookie button

    try: 
        cookie_button = driver_proxy.find_elements_by_xpath('//button[@class="confirm button radius margin-small secondary right"]')
        cookie_button[0].click()
        time.sleep(3 + random.uniform(-1,0))
#         driver_proxy.get(URL)

    except:
        pass
    
    
    r = driver_proxy.page_source
    
#     driver.quit()
    return bs(r, 'html.parser')  
#     return r


# In[11]:


def import_df(path, search_term): # takes in file path of df and search term. If search term is na.
    try: 
        df = pd.read_csv(path, index_col=0)
#         if 'overdrive_author' not in list(df.columns):
#             df['overdrive_author'] = np.nan
        list_of_indices = list(df[df[search_term].isna()].index)
    except FileNotFoundError:
        return 0, 0
    
    return list_of_indices, df


# In[3]:


def scrape_goodreads(url): # returns a df containing titles, ratings, urls, date published. DO NOT TOUCH!!
#     soup = get_soup_proxy(url)
    soup = get_soup_driver(url)
    list_results = []
    
    # get list of titles
    soup_titles = soup.find_all('a', class_='bookTitle')
    titles = [title.text.strip() for title in soup_titles]
    
    # get authors
    author_info = soup.find_all('a', class_='authorName')
    authors = [author.text for author in author_info]
    author_urls = [author.get('href') for author in author_info]
    
    # get list of ratings 
    stars = soup.find_all(class_='minirating')
    stars = [star.text.split('—') for star in stars]

    # some stars have messed up title starting with a string. Clean this up.
    for star in stars:
        s1 = star[0].strip()
        star[0] = star[0].strip()
        m = re.search(r"\d", s1)
        index = m.start()
        if index:
            star[0] = star[0][index:]

        
    # clean up extra white spaces   
    cleaned_stars = []
    for line in stars:
        transfer = []
        for item in line:
            transfer += [item.strip()]
        cleaned_stars += [transfer]

    
    
    # get year published
#     dates = soup.find_all(class_='minirating')
#     dates = [date.parent.text.split()[-1] for date in dates]
    
    # get url for GR page for specific title
    urls = [url.get('href') for url in soup_titles]
    
    #assemble list of dicts for each title
    for i in range(len(titles)):
        results = {}
        title = titles[i]
        results['title'] = title
        results['author'] = authors[i]
        results['author_url'] = author_urls[i]
        
        if cleaned_stars[i][0][0].isnumeric():       
            results['stars'] = cleaned_stars[i][0]
        else: results['stars'] = 'error'
        results['number of ratings'] = cleaned_stars[i][1]
#         results['year published'] = dates[i]
        results['url'] = urls[i]
        left = '(' in titles[i]
        right = ')' in titles[i]
        pound = '#' in titles[i]
        if left and right and pound:
            results['is_series'] = True
            
            # clean up title
            pound_ind = title.index('#')
            right_ind = title.index(')', pound_ind)
            title = title[::-1]
            left_ind = title.index('(', title.index('#'))
            left_ind = len(title) - left_ind - 1
            title = title[::-1]
            title = title[:left_ind] + title[right_ind + 1:]
            title = title.strip()
            results['title'] = title

        else: results['is_series'] = False


        list_results += [results]

    # convert list of dicts into df
    list_results = pd.DataFrame(list_results)
    
    # clean and return df
    return list_results


# In[7]:


def search_imdb(title): # get list of movies from IMDB with the title
    title = urllib.parse.quote(title, safe='~')
    url = f'https://www.imdb.com/search/title/?title={title}&title_type=feature'
    soup = get_soup(url)
    titles = soup.find_all('h3')
    result = {}
#     titles = [title.text.strip() for title in soup.find_all('h3')]
#     print(titles)
    if len(titles) > 1:
        result['movie title'] = titles[0].find('a').text.strip()
        result['movie date'] = titles[0].find(class_='lister-item-year').text[1:-1]
    else:
        result['movie title'] = False
        result['movie date'] = False

    return len(titles) > 1, result


# In[9]:


def get_pages_year_authorAttr(path): # pass a path to a df in. Go to the book url, get pages, year published
    
    list_of_indices, df = import_df_PYAA(path, 'Number of pages')
    base_url = 'https://www.goodreads.com'

    
    for index in list_of_indices:
#         print('ding')
        print(base_url + df.loc[index, 'url'])
        soup = get_soup_proxy(base_url + df.loc[index, 'url'])
        
        try: num_pages = soup.find(itemprop='numberOfPages').text
        except: num_pages = -1
        
        try: year_published = soup.find(text = re.compile('Published')).text.replace('Published','').replace('\n', '').strip()
        except: year_published = -1
            
        try: author_url = soup.find('a', class_='authorName').get('href')
        except: author_url = -1
            
        print(num_pages, year_published)
        
        time.sleep(11 + random.uniform(-1,1))
        
        if author_url != -1: soup = get_soup_proxy(author_url)

        try: rating = soup.find(class_='rating').text.replace('Average rating:', '').replace('\n','').strip()#replace
        except: rating = -1

        try: num_ratings = soup.find(class_='votes').text.replace('\n','').strip()#replace
        except: num_ratings = -1

        try: num_reviews = soup.find(class_='count').text.replace('\n','').strip()#replace
        except: num_reviews = -1

        try: num_distinct_works = soup.find(class_='hreview-aggregate').find('a').text
        except: num_distinct_works = -1           

        print(rating, num_ratings, num_reviews, num_distinct_works)
        
        df.loc[index, 'author_stars'] = rating      
        df.loc[index, 'author_ratings'] = num_ratings
        df.loc[index, 'author_num_reviews'] = num_reviews   
        df.loc[index, 'author_distinct_works'] = num_distinct_works
            
        
        df.loc[index, 'author_url'] = author_url      
        if df.loc[index, 'number of pages'] == '-1' or df.loc[index, 'number of pages'] == "'NoneType' object has no attribute 'parent'":
            df.loc[index, 'number of pages'] = num_pages
        
        if df.loc[index, 'year_published'] == '-1' or df.loc[index, 'year_published'] == "'NoneType' object has no attribute 'parent'":
            df.loc[index, 'year_published'] = year_published

        
        df.to_csv(path)
        time.sleep(11 + random.uniform(-1,1))
        
#     return pd.DataFrame(list_results)
    

def Ztest():
    print('test')
# In[ ]:






In [39]:
search_results = scrape_goodreads('https://www.goodreads.com/search?q=The%20Amazing%20Spider-Man%20by%20Nick%20Spencer%2C%20Volume%204%20Nick%20Spencer')

In [41]:
search_results

""


In [52]:
try: search_results['number of ratings'].apply(lambda x: float(x.split()[-2].replace(',', ''))).idxmax()
except: print ('err')

err


In [332]:
webdriver.Chrome()

<selenium.webdriver.chrome.webdriver.WebDriver (session="9484ad14832990cec98ccf5a4f2e4a42")>